In [1]:
import datetime
import ee
import pandas as pd
import sys
from IPython.display import Image
import geopandas as gpd 

sys.path.append(r'C:\openet_brazil\openet-geesebal')

import openet.geesebal as model

ee.Initialize()

In [2]:
collections = [ 
    'LANDSAT/LT04/C02/T1_L2',
    'LANDSAT/LT05/C02/T1_L2',
    'LANDSAT/LE07/C02/T1_L2',
    'LANDSAT/LC08/C02/T1_L2',
    'LANDSAT/LC09/C02/T1_L2']

# towers file
towers_info = gpd.read_file(r"C:\openet_brazil\flux-towers-data-br\location\FluxTowers_SA_2023.shp")

# Date range you want to aggregate ET over
start_date = '2006-01-01'
end_date = '2006-12-31'

# Only keep images with an average cloud cover less than
# Cloud cover filter parameter is not being passed in (yet)
cloud_cover = 30

# tower name
tower_name = 'BAN'

# get geometry
tower_location = towers_info[towers_info['site_id'] == tower_name.upper()]['geometry']

test_point = ee.Geometry.Point(tower_location.x.values[0], tower_location.y.values[0])
study_area = ee.Geometry.Rectangle(-122.00, 38.60, -121.00, 39.0)


In [9]:

model_obj = model.Collection(
    collections=collections,
    start_date=start_date,
    end_date=end_date,
    geometry=test_point,
    cloud_cover_max=30,
    model_args={
    'h_optmization':True,
    'meteo_location' :'br'
    # filter_args={},
    }
)

model_obj.get_image_ids()

['LANDSAT/LT05/C02/T1_L2/LT05_223067_20060129',
 'LANDSAT/LT05/C02/T1_L2/LT05_223067_20060419',
 'LANDSAT/LT05/C02/T1_L2/LT05_223067_20060606',
 'LANDSAT/LT05/C02/T1_L2/LT05_223067_20060622',
 'LANDSAT/LT05/C02/T1_L2/LT05_223067_20060708',
 'LANDSAT/LT05/C02/T1_L2/LT05_223067_20060724',
 'LANDSAT/LT05/C02/T1_L2/LT05_223067_20060809',
 'LANDSAT/LT05/C02/T1_L2/LT05_223067_20060825',
 'LANDSAT/LT05/C02/T1_L2/LT05_223067_20060910',
 'LANDSAT/LE07/C02/T1_L2/LE07_223067_20060121',
 'LANDSAT/LE07/C02/T1_L2/LE07_223067_20060614',
 'LANDSAT/LE07/C02/T1_L2/LE07_223067_20060630',
 'LANDSAT/LE07/C02/T1_L2/LE07_223067_20060716',
 'LANDSAT/LE07/C02/T1_L2/LE07_223067_20060801',
 'LANDSAT/LE07/C02/T1_L2/LE07_223067_20060817',
 'LANDSAT/LE07/C02/T1_L2/LE07_223067_20060902',
 'LANDSAT/LE07/C02/T1_L2/LE07_223067_20060918']

In [10]:
et_col = model_obj.overpass(variables=[ 'et'])


In [7]:

def get_table(img):
    
    reduce= img.reduceRegion(
        reducer=ee.Reducer.mean(),
        scale=90,
        maxPixels=1e12,
        geometry=test_point)

    date= ee.Date(img.get('system:time_start')).format('YYYY-MM-dd')
    
    return ee.Feature(None, {
      'date' : date,
      'et': ee.List([reduce.get('et'),-9999]).reduce(ee.Reducer.firstNonNull()),
    })
                                          


In [11]:
et_col.map(get_table).limit(1).getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'features': [{'type': 'Feature',
   'geometry': None,
   'id': '1_2_LT05_223067_20060129',
   'properties': {'date': '2006-01-29', 'et': 4.608010768890381}}]}

In [7]:
# Export the image sample feature collection to Drive as a CSV file.
task = ee.batch.Export.table.toDrive(
    collection=et_col.map(get_table),
    description='geesebal-h-opt-true-v2',
    folder='geesebal-python',
    fileFormat='CSV',
)
task.start()
